[Tensorboard pytorch tutorial](https://krishansubudhi.github.io/deeplearning/2020/03/24/tensorboard-pytorch.html)

Clear everything first 

In [60]:
! powershell "echo 'checking for existing tensorboard processes'"
! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}"

! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}| %{kill $_}"

! powershell "echo 'cleaning tensorboard temp dir'"
! powershell "rm -Force -Recurse $env:TEMP\.tensorboard-info\*" 

! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}"
! powershell "rm -Force -Recurse runs\*"

checking for existing tensorboard processes
cleaning tensorboard temp dir


# Create Summary writer

In [61]:
from torch.utils.tensorboard import SummaryWriter
# Writer will output to ./runs/ directory by default
writer = SummaryWriter('runs/tensorboard_bert')

# Load bert model

In [62]:
import transformers, torch
from transformers import BertForPreTraining, BertTokenizer
model = BertForPreTraining.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [63]:
#text = "I am using tensorboard in pytorch. I am trying tensorboard to visualize BERT model."
def get_tensors(text):
    tokens = tokenizer.tokenize(text)
    tokens.insert(0, tokenizer.cls_token)
    tokens.append(tokenizer.sep_token)

    correct_tensors = torch.LongTensor(tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)

    import numpy as np
    # Masking 20% except first and last
    for _ in range(len(tokens)//5):
        i = np.random.randint(1, len(tokens)-1)
        tokens[i] = tokenizer.mask_token


    tensors = torch.LongTensor(tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)
    #print(tokens)
    return  tokens, tensors, correct_tensors
#get_tensors(text)

# Adding model graph and embeddings 

In [64]:
writer.add_graph(model, torch.ones(1,128, dtype = torch.long))

# Adding scalars and histograms

In [65]:
def log_grads(step,model):
    print('Logging grads for ',step)
    for name,param in model.named_parameters():
        #print(name, param.grad.view(-1))
        if ('bert.encoder.layer' in name and 'bert.encoder.layer.11' not in name):
            continue
        flat_params =  param.view(-1).data.numpy()
        writer.add_histogram('weight/'+name, flat_params, step)
        if param.grad is not None:
            flat_params =  param.grad.view(-1).numpy()
            writer.add_histogram('grads/'+name, flat_params, step)
        
#log_grads(0,model)

In [66]:
from torch.optim import adam
optimizer = adam.Adam( model.parameters(),0.1)
def run_step(step, model, tensors, correct_tensors, nsp_labels):
    
    #forward
    loss,prediction_scores, nsp_score =  model.forward(tensors, masked_lm_labels = correct_tensors, next_sentence_label =nsp_labels )
    
    #scalars
    preds = torch.argmax(prediction_scores, dim = -1)
    acc = torch.sum(correct_tensors[:, 1:-1]==preds[:,1:-1]).item()/correct_tensors.size()[-1]
    writer.add_scalar('Loss', loss.item(), step)
    writer.add_scalar('Accuracy', acc, step)
    
    
    #backwars
    loss.backward()
    log_grads(step, model)
    
    optimizer.step()
    optimizer.zero_grad()
    
    return preds

In [67]:

texts = ["I am using tensorboard in pytorch. I am trying tensorboard to visualize BERT model.",
       "This is not scaling.",
       "Working from home these days because of lockdown.",
       "Hidden cup 3 for age of empires is awesome"]
for epoch in range(2):
    print('epoch',epoch)
    for i,text in enumerate(texts):
        print (text)
        tokens, tensors, correct_tensors = get_tensors(text)
        preds =  run_step(epoch*len(texts) + i, model, tensors, correct_tensors, nsp_labels = torch.ones(1,1,dtype = torch.long))
        predicted_tokens = tokenizer.convert_ids_to_tokens(preds.squeeze().numpy())
        print(tokens,'\n')
        print(predicted_tokens)
        print()

epoch 0
I am using tensorboard in pytorch. I am trying tensorboard to visualize BERT model.
Logging grads for  0
['[CLS]', 'i', 'am', 'using', 'tensor', '##board', 'in', 'p', '##yt', '##or', '##ch', '.', 'i', 'am', '[MASK]', '[MASK]', '##board', 'to', 'visual', '##ize', '[MASK]', '[MASK]', '.', '[SEP]'] 

['.', 'i', 'am', 'using', 'tensor', '##board', 'in', 'p', '##yt', '##or', '##ch', '.', 'i', 'am', 'using', 'tensor', '##board', 'to', 'visual', '##ize', 'the', 'objects', '.', '.']

This is not scaling.
Logging grads for  1
['[CLS]', 'this', '[MASK]', 'not', 'scaling', '.', '[SEP]'] 

['##ize', '##ize', '##ize', '##ize', '##ize', '##ize', '##ize']

Working from home these days because of lockdown.
Logging grads for  2
['[CLS]', 'working', '[MASK]', 'home', '[MASK]', 'days', 'because', 'of', 'lock', '##down', '.', '[SEP]'] 

['mac', 'mac', 'mac', 'mac', 'mac', 'mac', 'mac', 'mac', 'mac', 'mac', 'mac', 'mac']

Hidden cup 3 for age of empires is awesome
Logging grads for  3
['[CLS]', 'hi

In [68]:
writer.flush()

writer.close()



In [69]:

! pwd
! powershell "ls runs"
# Run in a new anaconda powershell. Change path for linux
# ! tensorboard --logdir="C:\Users\krkusuk\projects\tensorboard\runs"

'pwd' is not recognized as an internal or external command,
operable program or batch file.




    Directory: C:\Users\krkusuk\projects\tensorboard\runs


Mode                LastWriteTime         Length Name                          
----                -------------         ------ ----                          
d-----        3/25/2020  11:58 AM                tensorboard_bert              


